In [8]:
# from google.colab import drive
# drive.mount("/content/drive/", force_remount=True)

In [9]:
# import zipfile

# with zipfile.ZipFile("./drive/MyDrive/data.zip") as f:
#   f.extractall("")

In [10]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor,Resize,Compose
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

root = "./"
BATCH_SIZE = 300

train_ds = ImageFolder(root+"asl_train", Compose([Resize(128),ToTensor()]))
train_dl = DataLoader(train_ds, BATCH_SIZE, shuffle=True, pin_memory=True)

val_ds = ImageFolder(root+"asl_train", Compose([Resize(128),ToTensor()]))
val_dl = DataLoader(val_ds, 49, pin_memory=True)

print(len(train_ds))
img_cls = train_ds.classes 

CLS_SIZE = len(img_cls)

print(img_cls)
print(train_ds[0][0].shape)

310074
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
torch.Size([3, 128, 128])


In [11]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.cuda.empty_cache()

cuda


In [12]:
import torch.nn as nn


class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.network = nn.Sequential(
            # 3 128
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # 32 64
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            # 64 32
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # 128 16
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            # 128 8
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            # 256 4
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            # 256 2

            nn.Flatten(),
            # 256*2*2
            nn.Linear(256 * 2 * 2, 128),
            nn.ReLU(),
            # 128
            nn.Linear(128, 29)
            # 26
        )

    def forward(self, inputs):

        out = self.network(inputs)
        # return torch.softmax(out, dim=-1)
        return out

In [13]:
sign_model = Model().to(device)
# loss_fn = nn.BCELoss()
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.Adam(sign_model.parameters(),lr=0.001)

In [15]:
epochs = 10

for epoch in range(epochs):
    print("Epoch: ",epoch+1)
    correct = 0

    for images,labels in tqdm(train_dl):

        preds = sign_model(images.to(device))
        targets = torch.zeros(len(labels),CLS_SIZE, dtype=torch.float32)
        
        with torch.no_grad():
            for i in range(len(labels)):
                targets[i,labels[i]]=1

        loss = loss_fn(preds.to(device), targets.to(device))
        loss.backward()

        # check if the predicted class is correct compared to target, if it is then increment correct
        for i in range(len(labels)):
            if torch.argmax(preds[i]) == labels[i]:
                correct += 1
        
        opt.step()
        opt.zero_grad()

    print("Loss: ",round(loss.item(),6))

    accuracy = round(100*(correct/len(train_ds)), 2)
    print(f"Accuracy : {accuracy}%")

    torch.save(sign_model.state_dict(), f"sign_model_{round(accuracy)}.pth")

Epoch:  1


 32%|███▏      | 331/1034 [09:10<19:29,  1.66s/it]  


KeyboardInterrupt: 

In [ ]:
# code to reload the last saved model and then test it on the validation 